<center><h1>Gender Prediction using CNNs:<br>
    -- An Exploration of Image Recognition with InceptionV3 and Xception Models --</h1></center>

<p>
  <h2>Introduction:</h2><br>
  Image recognition stands as a pivotal application in the realm of Machine Learning, offering solutions for a diverse range of challenges such as security, object detection, face recognition, healthcare, and entertainment. Its potential to contribute significantly to societal well-being underscores the importance of exploring new applications, refining existing methodologies, and extracting more precise and valuable insights. An illustrative example of the impactful applications of image recognition is evident in the research conducted by The Chinese University of Hong Kong, where deep learning techniques were employed for face detection (<a href="https://arxiv.org/abs/1509.06451" target="_blank">source</a>).<br><br>

  In this project, we aim to construct a Machine Learning Algorithm utilizing Convolutional Neural Networks (CNNs) through pre-trained models, including Inception V3 and Xception. The primary objective is to predict the gender (male or female) of an individual based on input images, showcasing the versatility and efficacy of image classification techniques.
</p>

<h2>Dataset Overview:</h2>

<p>This project utilizes the <strong>men-women-classification</strong> dataset, accessible through Kaggle, and specifically focuses on the Men/Women Classification Dataset. The dataset is widely recognized in computer vision and deep learning circles, playing a crucial role in tasks such as face detection. It serves as an excellent resource for training and testing models designed to recognize facial attributes, including features like hair color, smiles, or the presence of glasses. The dataset encompasses a broad spectrum of challenges, including diverse poses, background variations, and a rich diversity of individuals. It comprises a substantial quantity of images accompanied by comprehensive annotations.</p>

<h3>Men / Women Classification Dataset:</h3>

<p>This manually curated and meticulously cleaned (<a href="https://www.kaggle.com/datasets/playlist/men-women-classification/data" target="_blank">men-women-classification</a>) dataset consists of 3,354 images (in JPG format), categorized into men (1,414 files) and women (1,940 files). The dataset aims to facilitate the development and evaluation of models focused on gender classification. Each image provides valuable insights into recognizing gender-related attributes, contributing to the broader field of face analysis.</p>

<h3>Validation Dataset:</h3>

<p>For validation purposes, a separate dataset is employed (<a href="https://www.kaggle.com/jessicali9530/celeba-dataset" target="_blank">celeba-dataset</a>), featuring 202,599 face images of numerous celebrities, spanning 10,177 unique identities. While the names of the identities are not disclosed, each image is annotated with 40 binary attributes and includes information about the locations of five facial landmarks. In the context of this project, the focus is specifically on utilizing the images from this dataset for validation purposes, omitting the additional attribute and landmark data.</p>

### Import Libraries

In [ ]:
# === Essential Libraries ===
# Importing necessary libraries for machine learning and visualization.
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import pandas as pd
import random 
import io
import cv2
from IPython.display import HTML, display
from PIL import Image
from io import BytesIO
import base64
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

# === Image Data Handling ===
# Utilizing modules for managing image data, including loading, converting, and augmenting.
from keras.utils import image_dataset_from_directory, load_img, img_to_array, plot_model
from keras.preprocessing.image import ImageDataGenerator

# === Model and Layer Components ===
# Setting up components related to building neural network models.
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.xception import Xception

# === Model Evaluation ===
# Utilizing modules for evaluation and testing metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, ConfusionMatrixDisplay

# === Random Seed Setting ===
# Ensuring reproducibility by setting random seeds.
initializer = glorot_uniform(seed=42)
np.random.seed(42)
tf.random.set_seed(42)

# === Explanation ===
# These sections import libraries, handle image data, define model components,
# optimize and regularize, and set random seeds for reproducibility.
# Each serves a crucial role in building and training deep learning models for image classification tasks.

### Initialize Variables

In [ ]:
IMG_WIDTH = 178
IMG_HEIGHT = 218
BATCH_SIZE = 16
NUM_EPOCHS = 20

### Define Functions for use

In [ ]:
def load_random_images(directory, num_images_per_class, class_name=['men', 'women']):
    """
    Load a specified number of random images from each class in a given directory using flow_from_directory.

    Parameters:
    - directory: Path to the directory containing the images.
    - num_images_per_class: Number of random images to load from each class.
    - Class_Name: List of class names.

    Returns:
    - Img: List of loaded images.
    - Label: List of corresponding labels.
    """

    # Creating an ImageDataGenerator for normalization.
    data_generator = ImageDataGenerator(rescale=1./255)

    # Lists to store loaded images and labels.
    img = []
    labels = []

    for class_name in class_name:
        class_directory = os.path.join(directory, class_name)

        # Collecting a list of image file names for the current class.
        image_file_names = os.listdir(class_directory)

        # Selecting num_images_per_class random images from the current class.
        selected_images = random.sample(image_file_names, num_images_per_class)

        for image_name in selected_images:
            image_path = os.path.join(class_directory, image_name)

            # Loading and processing the image.
            image = load_img(image_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
            image_array = img_to_array(image)
            label = class_name

            # Adding the loaded image and label to the lists.
            img.append(image_array)
            labels.append(label.title())

    return img, labels

In [ ]:
def plot_images_subplots(directory):
    """
    Plot images in a 4x4 grid of subplots using Plotly.

    Parameters:
    - Img: List of loaded images.
    - Label: List of corresponding labels.

    Returns:
    - None
    """
    # Grab images 
    num_images_per_class = 3
    Img, Label = load_random_images(directory, num_images_per_class)
    
    # Create a 4x4 grid of subplots
    fig = make_subplots(rows=2, cols=3, subplot_titles=[f"{i}" for i in Label], horizontal_spacing=0.02)

    s = 0
    for i in range(2):
        for j in range(3):
            # Add an image to each subplot
            fig.add_trace(go.Image(z=Img[s], dx=224, dy=224), row=i+1, col=j+1)
            s += 1

    # Hide x-axis tick labels
    fig.update_xaxes(showticklabels=False)
    # Hide y-axis tick labels
    fig.update_yaxes(showticklabels=False)
    
    # Update layout
    fig.update_layout(
        paper_bgcolor='#E7C18A',
        title_text='Gender Wise Images',
        title_x=0.5,  # Center the title horizontally
        title_y=0.95,  # Center the title vertically
        height=500
    )

    # Show the figure
    fig.show()

In [ ]:
def build_custom_model(base_model):
    """
    Build a custom model on top of a pre-trained base model for a specific task.

    Parameters:
    - base_model: The pre-trained base model.

    Returns:
    - model: The custom model.
    """

    # Freeze the layers of the pre-trained model
    for layer in base_model.layers[:52]:
        layer.trainable = False

    # Add custom layers for the specific task
    x = GlobalAveragePooling2D()(base_model.output)
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation="relu", kernel_initializer=initializer, kernel_regularizer=regularizers.l2(l2=0.005))(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation="relu", kernel_initializer=initializer, kernel_regularizer=regularizers.l2(l2=0.005))(x)
    x = Dense(148, activation="relu", kernel_initializer=initializer, kernel_regularizer=regularizers.l2(l2=0.005))(x)

    output = Dense(2, activation='softmax')(x)

    # Create the final model
    model = Model(inputs=base_model.input, outputs=output)

    # Compile the model with an appropriate optimizer and loss function
    model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def train_model(model, train_generator, valid_generator, save_model, epochs=NUM_EPOCHS):
    """
    Train a given model using the provided data generators.

    Parameters:
    - model: The model to be trained.
    - train_generator: Data generator for training data.
    - valid_generator: Data generator for validation data.
    - save_model: Path to save the best model checkpoint.
    - epochs: Number of training epochs (default is NUM_EPOCHS).

    Returns:
    - history: Training history.
    - model: Trained model.
    """

    # Define callbacks for model training
    checkpoint = ModelCheckpoint(save_model, save_best_only=True, monitor='val_loss', mode='min', verbose=1)
    # early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
    # reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

    # Train the model
    history = model.fit(
        train_generator,
        validation_data=valid_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_steps=valid_generator.samples // valid_generator.batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        verbose=1,
    )

    return history


In [ ]:
def plot_training_history(history):
    """
    Plot the training history, including accuracy and loss, over epochs.

    Parameters:
    - history: The training history obtained from model training.

    Returns:
    None
    """

    # Plot accuracy through epochs
    plt.figure(figsize=(18, 4))
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='valid')
    plt.legend()
    plt.title('Accuracy')
    plt.show()

    # Plot loss function value through epochs
    plt.figure(figsize=(18, 4))
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='valid')
    plt.legend()
    plt.title('Loss Function')
    plt.show()

In [ ]:
def img_to_display(filename):
    """
    Convert an image file to a base64-encoded string for displaying inline.

    Parameters:
    - filename (str): Path to the image file.

    Returns:
    - str: Base64-encoded string representing the image.
    """
    # Open the image using PIL
    image = Image.open(filename)
    
    # Resize the image to a thumbnail
    image.thumbnail((450, 450), Image.LANCZOS)

    # Save the image to a buffer in JPEG format
    with BytesIO() as buffer:
        image.save(buffer, 'jpeg')
        
        # Encode the image buffer to base64
        return base64.b64encode(buffer.getvalue()).decode()

In [ ]:
def get_random_image_with_prediction(model, dataset_path, target_size=(IMG_HEIGHT, IMG_WIDTH)):
    """
    Get the image array and prediction for a random image from a dataset.

    Parameters:
    - model (tf.keras.Model): The trained model for making predictions.
    - dataset_path (str): Path to the directory containing the images.
    - target_size (tuple): Size to resize the displayed image.

    Returns:
    - tuple: (image_array, prediction)
    """
    images, predictions = [], []
    for _ in range(3):
        # Get a list of all image files in the dataset directory
        image_files = [f for f in os.listdir(dataset_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

        if not image_files:
            print("No image files found in the specified directory.")
            return None

        # Select a random image file
        random_image_file = random.choice(image_files)
        random_image_path = os.path.join(dataset_path, random_image_file)

        # Open the image using PIL
        image = Image.open(random_image_path)

        # Resize the image to the target size
        image = image.resize(target_size)
        # Convert the image to a NumPy array
        image_array = img_to_array(image)
        images.append(random_image_path)
        image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
        image_array /= 255.0

        # Make a prediction using the model
        prediction = model.predict(image_array)[0]
        argmax = np.argmax(prediction)
        predictions.append(('Woman' if argmax == 1 else 'Man', prediction[argmax]))
    # Return the image array and prediction
    return images, predictions

In [ ]:
def display_images_with_predictions(images, predictions, card_width=300, border_radius=5):
    """
    Display images along with their predicted class and probability in a customizable card display.

    Parameters:
    - images: List of image paths or PIL Image objects.
    - predictions: List of tuples (class, probability) for each image.
    - card_width: Width of the card in pixels.
    - border_radius: Border radius for image corners in pixels.

    Returns:
    None
    """

    # Custom CSS for card display
    custom_css = f"""
    <style>
        .card {{
            border: 1px solid #ddd;
            padding: 10px;
            margin: 10px;
            text-align: center;
            width: {card_width}px;
            display: inline-block;
            border-radius: {border_radius}px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.3);
            transition: box-shadow 0.3s ease-in-out;
        }}

        .card:hover {{
            box-shadow: 0 8px 16px rgba(0, 0, 0, 0.2);
        }}

        .card img {{
            width: 100%;
            border-top-left-radius: {border_radius}px;
            border-top-right-radius: {border_radius}px;
        }}

        .prediction-info {{
            margin-top: 10px;
        }}
    </style>
    """

    display(HTML(custom_css))

    # Display images and predictions in cards
    for img, prediction in zip(images, predictions):
        # Display card with image, predicted class, and probability
        card_html = f"""
        <div class="card">
            <img src="data:image/jpeg;base64,{img_to_display(img)}" alt="Image" style="width:100%">
            <div class="prediction-info">
                <p style="font-weight: bold;">Predicted Class: {prediction[0]}</p>
                <p>Probability: {prediction[1]:.4f}</p>
            </div>
        </div>
        """

        display(HTML(card_html))

In [ ]:
def load_images_and_labels_for_testing(
    directory='/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba', 
    csv_path='/kaggle/input/celeba-dataset/list_attr_celeba.csv',
    target_size=(IMG_HEIGHT, IMG_WIDTH)):
    """
    Load images and corresponding labels from a directory and CSV file.

    Parameters:
    - directory: Path to the directory containing the images.
    - csv_path: Path to the CSV file with gender labels.

    Returns:
    - images: List of loaded images as numpy arrays.
    - labels: List of corresponding gender labels.
    """
    # Load CSV file
    df = pd.read_csv(csv_path)

    # Extract labels from the "Male" column
    labels = df['Male'].values[:1000]
    for l in range(len(labels)):
        labels[l] = 1 if labels[l] == -1 else 0
    
    # Lists to store loaded images and labels
    images = []
    
    # Sort the list of filenames
    file_list = sorted(os.listdir(directory))
    
    # Load images from the directory
    for i, filename in enumerate(file_list):
        img_path = os.path.join(directory, filename)
        im = cv2.imread(img_path)
        im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (IMG_WIDTH, IMG_HEIGHT)).astype(np.float32) / 255.0
        im = np.expand_dims(im, axis =0)
        images.append(im)
        if i == 999: break

    return images, labels

In [ ]:
def evaluate_model(model, x_test, y_test):
    """
    Evaluate a model using various metrics.

    Parameters:
    - model: Trained model to be evaluated.
    - x_test: List of images as numpy arrays.
    - y_test: List of corresponding labels.

    Returns:
    None
    """
    # Generate predictions
    model_predictions = [np.argmax(model.predict(img)[0]) for img in x_test]

    # Report test accuracy
    test_accuracy = accuracy_score(y_test, model_predictions)
    print('Model Evaluation:')
    print(f'Test accuracy: {test_accuracy:.4f}')

    # Confusion Matrix
    print('\nConfusion Matrix:')
    ConfusionMatrixDisplay.from_predictions(y_test, model_predictions, display_labels=['Men', 'Women'], cmap=plt.cm.Blues)

    # Classification Report
    print('\nClassification Report:')
    print(classification_report(y_test, model_predictions))

### Data Importation & Augmentation

<p>Data augmentation is applied using the `ImageDataGenerator` to enhance the diversity of the training dataset. Data augmentation involves performing various transformations on the existing images, such as rotation, shifting, shearing, zooming, and flipping. These augmented images are then used during training to expose the model to a wider range of variations, helping it generalize better to unseen data. The augmentation process aids in mitigating overfitting and improving the model's ability to handle diverse real-world scenarios by creating a more robust and varied training set.</p>

In [ ]:
# === Image Data Generators for Training and Validation ===

# Training Data Augmentation:
# - `rescale=1./255`: Normalize pixel values to the range [0, 1].
# - `rotation_range=30`: Randomly rotate images by up to 30 degrees.
# - `width_shift_range=0.2`: Randomly shift images horizontally by up to 20% of the width.
# - `height_shift_range=0.2`: Randomly shift images vertically by up to 20% of the height.
# - `shear_range=0.2`: Apply shear transformations with a maximum intensity of 20%.
# - `zoom_range=0.2`: Randomly zoom into images by up to 20%.
# - `horizontal_flip=True`: Randomly flip images horizontally.

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Validation Data Preprocessing:
# - `rescale=1./255`: Normalize pixel values to the range [0, 1].
valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# === Image Data Generators for Training and Validation Datasets ===

# Training Data Generator:
# - `flow_from_directory`: Generates batches of augmented training data from a directory.
# - `/kaggle/input/men-women-classification/data`: Path to the training data directory.
# - `target_size=(IMG_WIDTH, IMG_HEIGHT)`: Resizes images to the specified dimensions.
# - `class_mode='categorical'`: Uses categorical labels for multi-class classification.
# - `batch_size=BATCH_SIZE`: Number of samples per batch during training.
# - `seed=42`: Sets a seed for reproducibility.

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/men-women-classification/data',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    class_mode='categorical',
    batch_size=BATCH_SIZE, seed=42)

# Validation Data Generator:
# - `flow_from_directory`: Generates batches of validation data from a directory.
# - `/kaggle/input/menwomen-classification/testdata/testdata`: Path to the validation data directory.
# - `target_size=(IMG_WIDTH, IMG_HEIGHT)`: Resizes images to the specified dimensions.
# - `class_mode='categorical'`: Uses categorical labels for multi-class classification.
# - `batch_size=BATCH_SIZE`: Number of samples per batch during validation.
# - `seed=42`: Sets a seed for reproducibility.
# - `shuffle=False`: Disables shuffling to maintain order during evaluation.

valid_generator = valid_datagen.flow_from_directory(
    '/kaggle/input/menwomen-classification/testdata/testdata',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    class_mode='categorical',
    batch_size=BATCH_SIZE, seed=42, shuffle=False)

In [ ]:
train_generator.class_indices

In [ ]:
plot_images_subplots('/kaggle/input/men-women-classification/data')

### Gender Recognition - Build Model

#### 1. InceptionV3
The Inception-V3 model is a powerful convolutional neural network (CNN) designed for image classification tasks. Its architecture incorporates various convolutional and pooling layers, enabling it to capture intricate patterns and features in images. Below is a simplified visualization of the Inception-V3 model structure:
![](https://hackathonprojects.files.wordpress.com/2016/09/74911-image03.png)
**Source:** [https://hackathonprojects.files.wordpress.com/2016/09/74911-image03.png](https://hackathonprojects.files.wordpress.com/2016/09/74911-image03.png)



In [ ]:
# Load the InceptionV3 model pre-trained on ImageNet data
inception_model = InceptionV3(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights='imagenet')

# Build the custom model
incv3_model = build_custom_model(inception_model)

In [ ]:
# Train the model
inc_history = train_model(incv3_model, train_generator, valid_generator, 'inceptionV3_model.h5')

In [ ]:
# Evaluate model
incv3_model.evaluate(valid_generator)

* The model after NUM_EPOCHS got an accuracy over the validation data of **95.41%**.

In [ ]:
x_test, y_test = load_images_and_labels_for_testing()

In [ ]:
evaluate_model(incv3_model, x_test, y_test)

In [ ]:
# Plot model architecture
plot_model(
    incv3_model,
    to_file='incv3_model.png',
    show_shapes=True,
    show_dtype=False,
    show_layer_names=False,
    rankdir='TB',
    expand_nested=False,
    dpi=300,
    show_trainable=True,
    
)


In [ ]:
# Save model
incv3_model.save('/kaggle/working/inceptionv3.hdf5')

In [ ]:
plot_training_history(inc_history)

In [ ]:
images, predictions = get_random_image_with_prediction(incv3_model, '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba')

In [ ]:
display_images_with_predictions(images, predictions, card_width=200, border_radius=10)

### 2. Xception
The Xception model, an extension of the Inception architecture, stands out for its depth-wise separable convolutions. Developed to excel in image classification tasks, Xception enhances the efficiency of feature extraction and pattern recognition. Let's delve into the key components of the Xception model:
![](https://www.researchgate.net/profile/Abid_Mehmood3/publication/355098045/figure/fig2/AS:1076622409109511@1633698193851/Proposed-structure-of-Xception-network-used-within-each-stream-of-CNN.ppm)
**Source:** [https://www.researchgate.net/profile/Abid_Mehmood3/publication/355098045/figure/fig2/AS:1076622409109511@1633698193851/Proposed-structure-of-Xception-network-used-within-each-stream-of-CNN.ppm](https://www.researchgate.net/profile/Abid_Mehmood3/publication/355098045/figure/fig2/AS:1076622409109511@1633698193851/Proposed-structure-of-Xception-network-used-within-each-stream-of-CNN.ppm)

In [ ]:
# Load the InceptionV3 model pre-trained on ImageNet data
xc_model = Xception(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights='imagenet')

# Build the custom model
xception_model = build_custom_model(xc_model)

In [ ]:
# Train the model
xc_history = train_model(xception_model, train_generator, valid_generator, save_model='xception.h5')

In [ ]:
xception_model.evaluate(valid_generator)

* The model after NUM_EPOCHS got an accuracy over the validation data of **98.65%**.

In [ ]:
evaluate_model(xception_model, x_test, y_test)

In [ ]:
# Plot model architecture
plot_model(
    xception_model,
    to_file='xception_model.png',
    show_shapes=True,
    show_dtype=False,
    show_layer_names=False,
    rankdir='TB',
    expand_nested=False,
    dpi=300,
    show_trainable=True,
    
)


In [ ]:
xception_model.save('/kaggle/working/xception.hdf5')

In [ ]:
plot_training_history(xc_history)